# Лабораторная работа №5
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [7]:
def build_lps(pattern):
    lps = [0] * len(pattern)
    length = 0
    i = 1
    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps

def kmp_search(text, pattern):
    lps = build_lps(pattern)
    res = []
    i = j = 0
    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1
        if j == len(pattern):
            res.append(i - j)
            j = lps[j - 1]
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return res

# Основная часть программы
def main():
    text = input("Введите строку: ")
    pattern = input("Введите подстроку для поиска: ")
    case = input("Учитывать регистр? (да/нет): ").lower()

    if case == "нет":
        text = text.lower()
        pattern = pattern.lower()

    start = t.now()
    result = kmp_search(text, pattern)
    print(f"Позиции вхождения (KMP): {result}")
    print("Время работы:", (t.now() - start).total_seconds(), "сек")

if __name__ == "__main__":
    main()

Позиции вхождения (KMP): [0]
Время работы: 0.000202 сек


#### Упрощенный алгоритм Бойера-Мура

In [24]:
from datetime import datetime as t
from collections import defaultdict

def bm_search(text, pattern):
    last = defaultdict(lambda: -1)
    for i in range(len(pattern)):
        last[pattern[i]] = i

    res = []
    i = 0
    while i <= len(text) - len(pattern):
        j = len(pattern) - 1
        while j >= 0 and pattern[j] == text[i + j]:
            j -= 1
        if j < 0:
            res.append(i)
            i += 1
        else:
            shift = j - last[text[i + j]]
            i += max(1, shift)
    return res

# Основная часть программы
def main():
    text = input("Введите строку: ")
    pattern = input("Введите подстроку для поиска: ")
    case = input("Учитывать регистр? (да/нет): ").lower()

    if case == "нет":
        text = text.lower()
        pattern = pattern.lower()

    start = t.now()
    result = bm_search(text, pattern)
    print(f"Позиции вхождения (Boyer-Moore): {result}")
    print("Время работы:", (t.now() - start).total_seconds(), "сек")

if __name__ == "__main__":
    main()

Позиции вхождения (Boyer-Moore): [0]
Время работы: 0.000199 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [19]:
from heapq import heappop, heappush
from datetime import datetime as t

GOAL_STATE = tuple([1, 2, 3, 4,
                    5, 6, 7, 8,
                    9, 10, 11, 12,
                    13, 14, 15, 0])

MOVES = {
    'up': -4,
    'down': 4,
    'left': -1,
    'right': 1
}

def is_solvable(puzzle):
    inv_count = 0
    lst = list(puzzle)
    for i in range(15):
        for j in range(i + 1, 16):
            if lst[i] != 0 and lst[j] != 0 and lst[i] > lst[j]:
                inv_count += 1
    zero_row = 3 - (puzzle.index(0) // 4)
    return (inv_count + zero_row) % 2 == 0

def manhattan(state):
    distance = 0
    for i in range(16):
        if state[i] == 0:
            continue
        x, y = divmod(i, 4)
        goal_x, goal_y = divmod(state[i] - 1, 4)
        distance += abs(x - goal_x) + abs(y - goal_y)
    return distance

def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    x, y = divmod(zero_index, 4)
    for move, delta in MOVES.items():
        new_index = zero_index + delta
        if move == 'left' and y == 0:
            continue
        if move == 'right' and y == 3:
            continue
        if move == 'up' and x == 0:
            continue
        if move == 'down' and x == 3:
            continue
        new_state = list(state)
        new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
        neighbors.append((tuple(new_state), new_state[zero_index]))  # запоминаем какой элемент передвинули
    return neighbors

def solve(puzzle):
    start = t.now()
    if not is_solvable(puzzle):
        print("Решения нет.")
        return []

    open_set = []
    heappush(open_set, (manhattan(puzzle), 0, puzzle, []))
    visited = set()

    while open_set:
        f, g, state, path = heappop(open_set)
        if state == GOAL_STATE:
            print("Решение найдено за", t.now() - start)
            return path
        if state in visited:
            continue
        visited.add(state)
        for neighbor, moved_tile in get_neighbors(state):
            if neighbor not in visited:
                heappush(open_set, (
                    g + 1 + manhattan(neighbor),
                    g + 1,
                    neighbor,
                    path + [moved_tile]
                ))
    return []

user_input = input("Введите 16 чисел через пробел (0 — пустая клетка): ")
numbers = list(map(int, user_input.strip().split()))
if len(numbers) != 16:
    print("Ошибка: нужно ввести ровно 16 чисел.")
else:
    result = solve(tuple(numbers))
    if result:
        print("Решение найдено. Последовательность передвигаемых плиток:")
        print(result)
    else:
        print("Нет решения.")

Решение найдено за 0:00:00.000269
Решение найдено. Последовательность передвигаемых плиток:
[15]


### Вывод